## Lab 10 ##

In [122]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import image_dataset_from_directory
import pathlib
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
# installed pandas in this venv
import pandas as pd

I want to keep the only the 3 classes with the highest number of images. This information can be found in the attached birds.csv in the dataset.

In [2]:
metadata_df = pd.read_csv('/Users/karlokng/Downloads/data/birds_525/birds.csv')
print(metadata_df['class id'].value_counts().head(3))
print(metadata_df['labels'].value_counts().head(3))

class id
426.0    273
286.0    258
181.0    243
Name: count, dtype: int64
labels
RUFOUS TREPE        273
HOUSE FINCH         258
D-ARNAUDS BARBET    243
Name: count, dtype: int64


We see that the class with id 426, 286 and 181 are the classes with the highest number of photos.
These 3 class IDs correspond to the labels RUFOUS TREPE, HOUSE FINCH and D-ARNAUDS BARBET. As such, we'll keep these 3 folders in the data set, and drop everything else.

We note that the classes are relatively balanced, with the least (D_ARNAUDS BARBET) having 30 fewer images than the most (RUFOUS TREPE).

This was done manually using the system's file explorer.

The rest of the code is adapted from the in class 05C and 05D notebooks. Any changes would be pointed out in comments.

In [4]:
# define a function to augment data
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.2), # changed the percentages a little
        layers.RandomZoom(0.3),
        layers.RandomTranslation(height_factor=(-0.2, 0.2),
                                 width_factor=(-0.2, 0.2)) # translates images left/right/up/down
    ]
)

In [6]:
data_folder = pathlib.Path('/Users/karlokng/Downloads/data/birds_525')

train_dataset = image_dataset_from_directory(
    data_folder / "train",
    image_size=(224, 224), # image_size is changed to the image size of the data
    batch_size=32)
validation_dataset = image_dataset_from_directory(
    data_folder / "valid",
    image_size=(224, 224),
    batch_size=32)
test_dataset = image_dataset_from_directory(
    data_folder / "test",
    image_size=(224, 224),
    batch_size=32)

Found 744 files belonging to 3 classes.
Found 15 files belonging to 3 classes.
Found 15 files belonging to 3 classes.


In [8]:
import numpy as np

conv_base = keras.applications.vgg16.VGG16(
    weights="imagenet",
    include_top=False,
    input_shape=(224, 224, 3)) # input_shape change to accomodate change in image size
conv_base.trainable = False

def get_features_and_labels(dataset):
    all_features = []
    all_labels = []
    for images, labels in dataset:
        preprocessed_images = keras.applications.vgg16.preprocess_input(images)
        features = conv_base.predict(preprocessed_images)
        all_features.append(features)
        all_labels.append(labels)
    return np.concatenate(all_features), np.concatenate(all_labels)

train_features, train_labels =  get_features_and_labels(train_dataset)
val_features, val_labels =  get_features_and_labels(validation_dataset)
test_features, test_labels =  get_features_and_labels(test_dataset)

1/1 [==============================] - 0s 245ms/step


2023-12-02 16:27:07.156966: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 19ms/step


In [12]:
train_features.shape # double check features shape

(744, 7, 7, 512)

In [147]:
# define the dense layers at the end
inputs = keras.Input(shape=(224, 224, 3)) # change input shape to reflect train_dataset
x = data_augmentation(inputs)
x = keras.applications.vgg16.preprocess_input(x)
x = conv_base(x)
x = layers.Flatten()(x)

# adding 3 layers of Dense nodes, emulating the original structure of VGG16
x = layers.Dense(4096)(x)
x = layers.Dense(4096)(x)
x = layers.Dense(1000)(x)
x = layers.Dropout(0.5)(x)

# change output layers to 3 as we have 3 classes
# activation = "softmax" as we are working with 3 classes as well
outputs = layers.Dense(3, activation="softmax")(x)
model = keras.Model(inputs, outputs)

model.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_15 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 sequential_1 (Sequential)   (None, None, None, None   0         
                             )                                   
                                                                 
 tf.__operators__.getitem_7  (None, 224, 224, 3)       0         
  (SlicingOpLambda)                                              
                                                                 
 tf.nn.bias_add_7 (TFOpLamb  (None, 224, 224, 3)       0         
 da)                                                             
                                                                 
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                           

In [27]:
set(train_labels) # we see that our labels are not one-hot encoded

{0, 1, 2}

Since our labels are not one-hot encoded, we would use sparse_categorical_crossentropy.

In [148]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

callbacks = [
    keras.callbacks.ModelCheckpoint(
      filepath="./models/vgg16_bird_classifier.tf",
      save_best_only=True,
      monitor="val_loss")
]
history = model.fit(
    train_dataset,
    epochs=20,
    validation_data= validation_dataset,
    callbacks=callbacks)

Epoch 1/20


2023-12-02 22:00:23.084081: I metal_plugin/src/kernels/stateless_random_op.cc:282] Note the GPU implementation does not produce the same series as CPU implementation.


 1/24 [>.............................] - ETA: 1:07 - loss: 27.8413 - accuracy: 0.3438

2023-12-02 22:00:25.044302: I metal_plugin/src/kernels/stateless_random_op.cc:282] Note the GPU implementation does not produce the same series as CPU implementation.


 2/24 [=>............................] - ETA: 15s - loss: 2605.4153 - accuracy: 0.3281

...

22/24 [==========================>...] - ETA: 1s - loss: 1699.2477 - accuracy: 0.9418

2023-12-02 22:05:58.756537: I metal_plugin/src/kernels/stateless_random_op.cc:282] Note the GPU implementation does not produce the same series as CPU implementation.


23/24 [===========================>..] - ETA: 0s - loss: 1625.3673 - accuracy: 0.9443

2023-12-02 22:05:59.421967: I metal_plugin/src/kernels/stateless_random_op.cc:282] Note the GPU implementation does not produce the same series as CPU implementation.


24/24 [==============================] - 17s 698ms/step - loss: 1607.8903 - accuracy: 0.9449 - val_loss: 0.0000e+00 - val_accuracy: 1.0000


In [184]:
test_model = keras.models.load_model(
    "./models/vgg16_bird_classifier.tf")
test_loss, test_acc = test_model.evaluate(test_dataset)

# we need to change the encoding of the labels to feed into confusion matrix
def change_encoding(labels):
    output = []
    for row in labels:
        # the argmax would be the corresponding class
        label = np.argmax(row)
        output.append(label)
    return output

conf_mat_lst = []
precision_lst = []
recall_lst = []
roc_auc_lst = []
for i in range(0, 10):
    pred_vals = change_encoding(test_model.predict(test_dataset))

    # confusion matrix is in the form:
    #  actual\predicted  0     1     2
    #            0       
    #            1       
    #            2       
    # where the row labels are (actual) and column labels are (predicted)
    conf_mat = confusion_matrix(test_labels, pred_vals)

    # I chose 'macro' as it
    precision_macro = precision_score(test_labels, pred_vals, average= 'macro')
    recall_macro = recall_score(test_labels, pred_vals, average= 'macro')

    # the roc_auc_score requires the probability for each class, so we feed in the 
    # raw model.predict results instead of my converted ones
    # multi_class = 'ovr' as our classes are relatively balanced
    roc_auc = roc_auc_score(test_labels, test_model.predict(test_dataset), multi_class= 'ovr')

    conf_mat_lst.append(conf_mat)
    precision_lst.append(precision_macro)
    recall_lst.append(recall_macro)
    roc_auc_lst.append(roc_auc)

1/1 [==============================] - 0s 14ms/step


In [186]:
print(np.mean(precision_lst))
print(np.mean(recall_lst))
print(np.mean(roc_auc_lst))
print(conf_mat_lst)

0.37333333333333335
0.37333333333333335
0.44499999999999995
[array([[1, 3, 1],
       [2, 1, 2],
       [2, 1, 2]]), array([[2, 1, 2],
       [3, 0, 2],
       [0, 4, 1]]), array([[2, 2, 1],
       [1, 2, 2],
       [2, 1, 2]]), array([[3, 1, 1],
       [2, 2, 1],
       [0, 2, 3]]), array([[2, 2, 1],
       [2, 1, 2],
       [1, 2, 2]]), array([[1, 2, 2],
       [1, 2, 2],
       [3, 1, 1]]), array([[2, 2, 1],
       [2, 2, 1],
       [1, 1, 3]]), array([[2, 2, 1],
       [3, 1, 1],
       [0, 2, 3]]), array([[2, 2, 1],
       [2, 1, 2],
       [1, 2, 2]]), array([[3, 0, 2],
       [1, 3, 1],
       [1, 2, 2]])]


Note: Our model seems to behave non-deterministicly. I'm not entirely sure why that is at the moment, as the non-deterministic element (the dropout layer) should be disabled when running the model.predict() line. As such, I have decided to run the prediction 10 times, and get the mean performance metric value.

Our model does not perform very well at classifying these birds at the moment. Both precision and recall are very low, and the AUC value is lower than 0.5. This indicates that our model performs worse than random chance.

This can also be seen by the precision and recall values. They are both quite low at around 0.37. This indicates that it does not distinguish between the different species of birds very well.

Looking at the confusion matrix, there seems to be a relatively even number of errors across the matrix. However, it is only 15 photographs in the test set, which is a very limited set. As such, small deviations in performance would have a great impact on our metrics given that a lot of them are proportional. As such, a larger test set would more adequately demonstrate the model's performance.